In [30]:
import os
import json
import pandas as pd
while os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [31]:
reduced_modes = [False]

In [32]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from google.cloud import storage

In [33]:
for reduced_mode in reduced_modes:
    data = pd.read_pickle('data/combined_mouse_human_data.pkl')
    data.environment.unique()
    environment_indices = data.environment
    predictor_indices = data.irradiated
    if 'irradiated' in data.columns:
        data = data[data.columns[11:-1]]
    if reduced_mode:
        top_1000_variance_columns = list(data.std().sort_values(ascending=False).iloc[:1000].index)
        data = data[top_1000_variance_columns]
        
    data['Target'] = predictor_indices
    data['environment'] = environment_indices
    

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('ah21_data')
    if reduced_mode:
        experiment_directory = 'final_gamma_ray_experiment_reduced/'
    else: 
        experiment_directory = 'final_gamma_ray_experiment/'

    for seed in tqdm(range(0,75)):
        seed_directory = 'seed_{}/'.format(str(seed))

        for index, environment in enumerate(data.environment.unique()):
#             colab_directory = 'col_{}_{}/'.format(index, environment.replace(" ", ""))
            colab_directory = 'col_{}/'.format(index)
            env_data = data[data.environment == environment]
            env_data = env_data.drop('environment', axis=1)
            train, test = train_test_split(env_data, test_size=0.2, random_state = seed)

            blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'train/data.csv')
            train.to_csv('data_gamma_ray.csv')
#             print(train.shape)
            blob.upload_from_filename('data_gamma_ray.csv')

            blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'test/data.csv')
            test.to_csv('data_gamma_ray.csv')
#             print(test.shape)
            blob.upload_from_filename('data_gamma_ray.csv')


100%|██████████| 75/75 [32:48<00:00, 26.25s/it]


In [34]:
import streamlit as st

In [9]:
st.bar_chart()

array(['GLDS152femaleHom', 'GLDS152maleHom', 'GLDS157maleHom',
       'GLDS157femaleHom', 'GLDS156maleMus', 'GSE124612maleMus',
       'GSE62623maleMus'], dtype=object)

In [36]:
import plotly.express as px

In [37]:
data_canada = px.data.gapminder().query("country == 'Canada'")

In [38]:
px.bar()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
240,Canada,Americas,1952,68.750,14785584,11367.16112,CAN,124
241,Canada,Americas,1957,69.960,17010154,12489.95006,CAN,124
242,Canada,Americas,1962,71.300,18985849,13462.48555,CAN,124
243,Canada,Americas,1967,72.130,20819767,16076.58803,CAN,124
244,Canada,Americas,1972,72.880,22284500,18970.57086,CAN,124
245,Canada,Americas,1977,74.210,23796400,22090.88306,CAN,124
246,Canada,Americas,1982,75.760,25201900,22898.79214,CAN,124
247,Canada,Americas,1987,76.860,26549700,26626.51503,CAN,124
248,Canada,Americas,1992,77.950,28523502,26342.88426,CAN,124
249,Canada,Americas,1997,78.610,30305843,28954.92589,CAN,124


In [12]:
for seed in tqdm(range(0,75)):
    seed_directory = 'seed_{}/'.format(str(seed))
    
    for index, environment in enumerate(data.environment.unique()):
        colab_directory = 'col_{}_{}/'.format(index, environment.replace(" ", ""))
        env_data = data[data.environment == environment]
        env_data = env_data.drop('environment', axis=1)
        train, test = train_test_split(env_data, test_size=0.2, random_state = seed)
        
        blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'train/data.csv')
        train.to_csv('data.csv')
        blob.upload_from_filename('data.csv')

        blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'test/data.csv')
        test.to_csv('data.csv')
        blob.upload_from_filename('data.csv')

100%|██████████| 75/75 [14:59<00:00, 11.99s/it]


In [55]:
top_1000_variance_columns = list(data.std().sort_values(ascending=False).iloc[:1000].index)

In [69]:
column_indices = [data.columns.get_loc(col_name) for col_name in top_1000_variance_columns]

In [75]:
import pickle as pkl

In [76]:
# with open('data/column_names.pkl', 'wb') as handle:
#     pkl.dump(top_1000_variance_columns, handle)

In [77]:
# with open('data/column_indices.pkl', 'wb') as handle:
#     pkl.dump(column_indices, handle)

In [78]:
# from google.cloud import storage
# storage_client = storage.Client()
# bucket = storage_client.get_bucket('ah21_data')
# experiment_directory = 'gamma_ray_experiment_keys/'

In [79]:
# blob = bucket.blob('gamma_ray_experiment_keys/column_names.pkl')
# blob.upload_from_filename('data/column_names.pkl')

In [51]:
# data.std().sort_values(ascending=False).iloc[:1000].index

Index(['otof', 'ankk1', 'ildr1', 'alas2', 'hba1', 'cxcr4', 'tmsb4y', 'exosc2',
       'ier2', 'fech',
       ...
       'ubtf', 'reg1a', 'fkbp8', 'fth1', 'acin1', 'dgkd', 'phlda3', 'pgm2',
       'agmat', 'lyst'],
      dtype='object', length=1000)

In [52]:
data[top_1000_variance_columns]

,otof,ankk1,ildr1,alas2,hba1,cxcr4,tmsb4y,exosc2,ier2,fech,...,ubtf,reg1a,fkbp8,fth1,acin1,dgkd,phlda3,pgm2,agmat,lyst
0,0.138890,4.360115,4.361449,-0.058130,0.231537,4.133728,-0.254508,-0.289482,4.070327,-0.237934,...,-0.263666,-0.294737,-0.261767,3.967631,-0.157083,0.758235,-0.179378,-0.127901,-0.279839,0.404100
1,0.167062,4.715629,4.704571,-0.105785,0.809022,4.561452,-0.232294,-0.253940,4.311615,-0.227548,...,-0.222584,-0.263727,-0.238892,4.216564,-0.160504,0.724128,-0.156731,-0.155554,-0.252914,0.494188
2,0.102245,4.569738,4.526265,1.951066,2.358859,4.097987,-0.224929,-0.238978,3.768444,-0.217838,...,-0.222455,-0.267945,-0.244958,3.846191,-0.162600,0.423660,-0.160361,-0.149099,-0.257365,0.667701
3,0.085518,4.615641,4.615707,0.740284,1.014530,4.611963,-0.240857,-0.263481,4.487467,-0.238528,...,-0.249439,-0.286200,-0.259839,4.090940,-0.184769,0.742988,-0.274475,-0.127241,-0.261247,0.703017
4,0.061937,4.866759,4.702637,0.801783,0.958405,4.178572,-0.230302,-0.262449,4.411400,-0.249106,...,-0.239283,-0.270507,-0.253098,4.083599,0.001892,0.392296,-0.203293,-0.047260,-0.266113,0.704536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,4.872523,-0.225108,-0.230612,4.398118,5.162284,-0.225149,3.039074,3.034621,-0.226324,2.921544,...,1.449565,-0.230143,-0.096875,3.624910,-0.084885,-0.187953,-0.161970,0.574771,-0.190010,-0.229564
310,5.054484,-0.161292,-0.170600,5.001897,5.553835,-0.163776,3.214877,3.676661,-0.168171,3.257636,...,0.403751,-0.170563,0.035092,3.738239,-0.011272,-0.088847,0.002194,-0.086826,-0.068053,-0.164744
311,4.866630,-0.256856,-0.265660,4.787391,5.379770,-0.152859,2.876590,4.418721,-0.243465,3.790863,...,0.389136,-0.265602,0.034990,3.957975,-0.001211,-0.047510,0.119220,-0.082601,-0.145203,-0.231814
312,4.958770,-0.165538,-0.172764,5.046223,5.543773,-0.156746,3.001777,3.650067,-0.168866,3.231382,...,0.264554,-0.171267,0.002163,3.569402,-0.001299,-0.103978,-0.022443,-0.044766,-0.011936,-0.167901
